In [ ]:
pip install imblearn

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from sklearn import preprocessing
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score
import seaborn as sns
import imblearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint

#import data
data = pd.read_csv('data.csv')
data.head()

questions = pd.read_csv('questions.csv')
questions.describe()

questions.head()

data.describe()

data0 = pd.read_csv("neededcode.csv")

data0.describe()

ohe = OneHotEncoder(sparse=False)

ohe.fit_transform(data0[['gender']])

ohe.categories_

#checking for missing values
x = data0.isnull().sum()
set(x)

#Removing data that has zero accuracy
data0 = data0[data.accuracy != 0]

modified_values = data[data.age<120].age
print(modified_values.mean())
print(modified_values.median())
print(modified_values.mode()[0])

age_outliers = []

for value in data0.age.unique():
    if value > 120: 
        age_outliers.append(value)
#These are clearly outliers since the oldest human know lived to 120; this is clearly an anomoly
#currently replacing with median of age column 
age_median = data0.age.median()
data0.age = np.where(data0.age > 120, age_median, data0.age)

data0.age.describe()

gender_mode = data0.gender.mode() #gender can only be 1,2,3 not 0
data0.gender = np.where(data0.gender == 0, gender_mode, data0.gender)

#replace the rows where affiliative score is 5.1
data0 = data0[data.affiliative <5.1]

data0.describe()

#male = np.where(data0.gender == 1)
male = data0[data0.gender == 1].value_counts()
print(male)
female = data0[data0.gender == 2].value_counts()
print(female)
others = data0[data0.gender == 3].value_counts()
print(others)

from collections import Counter
import collections
indexs_with_minus = []

data["mod_ac"] = pd.cut(data.accuracy, bins = [0,10,20,30,40,50,60,70,80,90,100])

for value in data:
    a = list(data[data[value] == float(-1)].index)
    indexs_with_minus.extend(a)


accuracy_list = []

for value in set(indexs_with_minus):
      accuracy_list.append(data.iloc[value].mod_ac)
Counter(accuracy_list)


plt.title('Age')
data0.age.hist()

plt.title('Affiliative Score')
data0.affiliative.hist()

plt.title('Selfenhancing Score')
data0.selfenhancing.hist()

plt.title('Aggressive Score')
data0.agressive.hist()

plt.title('Selfdefeating Score')
data0.selfdefeating.hist()

plt.title('Gender')
data0.gender.hist()

plt.title('Accuracy')
data0.accuracy.hist()

plt.title('Accuracy')
sns.violinplot(data = data0.accuracy,size=4)
plt.grid() 

plt.title('Age')
sns.violinplot(data = data0.age,size=4)
plt.grid() 

plt.title('Affiliative Score')
sns.violinplot(data = data0.agressive,size=4)
plt.grid() 

plt.title('Agressive Score')
sns.violinplot(data = data0.agressive,size=4)
plt.grid()

plt.title('Selfdefeating Score')
sns.violinplot(data = data0.selfdefeating,size=4)
plt.grid()

plt.title('Selfenhancing Score')
sns.violinplot(data = data0.selfenhancing,size=4)
plt.grid()

plt.title('Gender')
sns.violinplot(data = data0.gender,size=4)
plt.grid() 

sns.set_style("whitegrid")
sns.pairplot(data0,hue = "gender", height = 3)

corrMatrix = data0.corr()
sns.heatmap(corrMatrix, annot=True)
plt.show()

#remove outliers for affiliative score
Q1_1 = data.affiliative.quantile(0.25)
Q3_1 = data.affiliative.quantile(0.75)
IQR_1 = Q3_1 - Q1_1
lower_limit_1 = Q1_1 - 1.5*IQR_1
upper_limit_1 = Q3_1 + 1.5*IQR_1
print(lower_limit_1,upper_limit_1)
#data0[(data0.affiliative<lower_limit_1)|data0.affiliative>upper_limit_1]
data0 = data0[(data0.affiliative>lower_limit_1)&(data0.affiliative<upper_limit_1)]
#data0.describe()
data0.describe()

#remove outliers for selfenhancing score
Q1_2 = data.selfenhancing.quantile(0.25)
Q3_2 = data.selfenhancing.quantile(0.75)
IQR_2 = Q3_2 - Q1_2
lower_limit_2 = Q1_2 - 1.5*IQR_2
upper_limit_2 = Q3_2 + 1.5*IQR_2
print(lower_limit_2,upper_limit_2)
data0 = data0[(data0.selfenhancing>lower_limit_2)&(data0.selfenhancing<upper_limit_2)]
data0.describe()
#data_no_outlier = data0[(data0.affiliative>lower_limit_1)&data0.affiliative<upper_limit_1]
#data_no_outlier.describe()

#remove outliers for agressive score
Q1_3 = data.agressive.quantile(0.25)
Q3_3 = data.agressive.quantile(0.75)
IQR_3 = Q3_3 - Q1_3
lower_limit_3 = Q1_3 - 1.5*IQR_3
upper_limit_3 = Q3_3 + 1.5*IQR_3
print(lower_limit_3,upper_limit_3)
data0 = data0[(data0.agressive>lower_limit_3)&(data0.agressive<upper_limit_3)]
data0.describe()
#data_no_outlier = data0[(data0.affiliative>lower_limit_1)&data0.affiliative<upper_limit_1]
#data_no_outlier.describe()

#remove outliers for agressive score
Q1_4 = data.selfdefeating.quantile(0.25)
Q3_4 = data.selfdefeating.quantile(0.75)
IQR_4 = Q3_4 - Q1_4
lower_limit_4 = Q1_4 - 1.5*IQR_4
upper_limit_4 = Q3_4 + 1.5*IQR_4
print(lower_limit_4,upper_limit_4)
data0 = data0[(data0.selfdefeating>lower_limit_4)&(data0.selfdefeating<upper_limit_4)]
data0.describe()
#data_no_outlier = data0[(data0.affiliative>lower_limit_1)&data0.affiliative<upper_limit_1]
#data_no_outlier.describe()

# Create X (all the feature columns)
#X = data.drop(["gender"],axis=1)
# Create y (the target column)
#y = data[["gender"]]
#data.columns
X = data0[['affiliative', 'selfenhancing', 'agressive', 'selfdefeating', 'age','accuracy']]
Y = data0['gender']

X

Y
#y = label_binarize(y, classes=[0, 1, 2])
#n_classes = y.shape[1]


#split into train and test values, using minmax scaled data
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size = 0.2)

X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

#Apply SMOTE to balance gender
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
Xtrain_smote, Ytrain_smote = oversample.fit_resample(X_train.astype('int'),Y_train)
from collections import Counter
print("Y Before SMOTE:", Counter(Y_train))
print("Y After SMOTE:", Counter(Ytrain_smote))

Xtrain_smote.shape, Ytrain_smote.shape

# StandardScalar
StandardScaler = preprocessing.StandardScaler()
Standard_scaled = StandardScaler.fit_transform(X)
Standard_scaled_data = pd.DataFrame(Standard_scaled,columns = X.columns.values,index=X.index)
Standard_scaled_data

#MinMaxScaler
MinMaxScaler = preprocessing.MinMaxScaler(feature_range=(0,1))
MinMax_scaled = MinMaxScaler.fit_transform(X)
MinMax_scaled_data = pd.DataFrame(MinMax_scaled,columns = X.columns.values,index = X.index)
MinMax_scaled_data

#Robust Scaler
Robust_Scaler = preprocessing.RobustScaler()
Robust_Scaled = Robust_Scaler.fit_transform(X)
Robust_Scaled_data = pd.DataFrame(Robust_Scaled,columns = X.columns.values,index = X.index)
Robust_Scaled_data

fig,(ax1,ax2,ax3,ax4) = plt.subplots(ncols=4,figsize=(30,10))
ax1.set_title('Before Scaling')
sns.kdeplot(X_smote['affiliative'], ax=ax1)
sns.kdeplot(X_smote['selfenhancing'], ax=ax1)
sns.kdeplot(X_smote['agressive'], ax=ax1)
sns.kdeplot(X_smote['selfdefeating'], ax=ax1)
sns.kdeplot(X_smote['age'], ax=ax1)
sns.kdeplot(X_smote['accuracy'], ax=ax1)


ax2.set_title('After Standard Scaler')
sns.kdeplot(Standard_scaled_data['affiliative'], ax=ax2)
sns.kdeplot(Standard_scaled_data['selfenhancing'], ax=ax2)
sns.kdeplot(Standard_scaled_data['agressive'], ax=ax2)
sns.kdeplot(Standard_scaled_data['selfdefeating'], ax=ax2)
sns.kdeplot(Standard_scaled_data['age'], ax=ax2)
sns.kdeplot(Standard_scaled_data['accuracy'], ax=ax2)

ax3.set_title('After Normalization/MinMax Scaler')
sns.kdeplot(MinMax_scaled_data['affiliative'], ax=ax3)
sns.kdeplot(MinMax_scaled_data['selfenhancing'], ax=ax3)
sns.kdeplot(MinMax_scaled_data['agressive'], ax=ax3)
sns.kdeplot(MinMax_scaled_data['selfdefeating'], ax=ax3)
sns.kdeplot(MinMax_scaled_data['age'], ax=ax3)
sns.kdeplot(MinMax_scaled_data['accuracy'], ax=ax3)

ax4.set_title('After Robust Scaler')
sns.kdeplot(Robust_Scaled_data['affiliative'], ax=ax4)
sns.kdeplot(Robust_Scaled_data['selfenhancing'], ax=ax4)
sns.kdeplot(Robust_Scaled_data['agressive'], ax=ax4)
sns.kdeplot(Robust_Scaled_data['selfdefeating'], ax=ax4)
sns.kdeplot(Robust_Scaled_data['age'], ax=ax4)
sns.kdeplot(Robust_Scaled_data['accuracy'], ax=ax4)
plt.show()

#apply min-max scaler to scale the train and test data
MinMaxScaler = preprocessing.MinMaxScaler(feature_range=(0,1))
MinMax_scaled_X_train = MinMaxScaler.fit_transform(Xtrain_smote)
#Robust_Scaled_data = pd.DataFrame(Robust_Scaled,columns = X.columns.values,index = X.index)
#MinMax_scaled_data = pd.DataFrame(MinMax_scaled_X_train,columns = MinMax_scaled_X_train.columns,index = MinMax_scaled_X_train)
#MinMax_scaled_data
MinMax_scaled_X_test = MinMaxScaler.fit_transform(X_test)

MinMax_scaled_X_train.shape

#################################################
#First we test on standard situation, where we care all three class equally or expecting the third class to have a higher accuracy
#we have up sampled and scale on X training set, and upsampled on Y traning set, and scaled on the X test set. 
#







########## Define a evlauation function for hyperparameter tuning 
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

#predict using random forest classifier
#standard method without using any corss validation or hyperparameters
from sklearn.ensemble import RandomForestClassifier

# Setup random seed
np.random.seed(42)
# Instantiate and fit the model (on the training set)
RFC = RandomForestClassifier()
RFC.fit(MinMax_scaled_X_train, Ytrain_smote.values.ravel())
# Check the score of the model (on both training and test set)
print(f"Random Forest Classifier Accuracy on training set: {RFC.score(MinMax_scaled_X_train, Ytrain_smote) * 100:.2f}%")
print(f"Random Forest Classifier Accuracy: {RFC.score(MinMax_scaled_X_test, Y_test) * 100:.2f}%")

###Check the default parameters

print('Parameters currently in use:\n')
pprint(RFC.get_params())

########## apply random hyperparameter search to 
# find a rough range of best parameters
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

# Use the random grid to search for best hyperparameters
# First create the base model to tune
RFC_test = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
RFC_test_random = RandomizedSearchCV(estimator = RFC_test, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
RFC_test_random.fit(MinMax_scaled_X_train, Ytrain_smote.values.ravel())

RFC_test_random.best_params_

##Evaluate the result of randomsearchcv

best_random = RFC_test_random.best_estimator_
random_accuracy = evaluate(best_random, MinMax_scaled_X_test, Y_test)

#apply cross validation with 10 folds and Grid Search for hyperparameters
#grid selection based on random search cv
param_grid = {
    'bootstrap': [True,False],
    'max_depth': [10, 30, 60, 90],
    'max_features': [2, 3, 'sqrt'],
    'min_samples_leaf': [1 , 3],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 500, 1000, 1800]
}

RFC_tuned = RandomForestClassifier()
grid_search_RFC = GridSearchCV(estimator = RFC_tuned,param_grid=param_grid,
                           cv = 10, n_jobs = -1, verbose = 2)



grid_search_RFC.fit(MinMax_scaled_X_train, Ytrain_smote.values.ravel())
grid_search_RFC.best_params_


################## Check the best hyperparameter score

best_grid_RFC = grid_search_RFC.best_estimator_
grid_training_accuracy_RFC = evaluate(best_grid_RFC, MinMax_scaled_X_train, Ytrain_smote.values.ravel())
grid_accuracy_RFC = evaluate(best_grid_RFC, MinMax_scaled_X_test, Y_test)
grid_search_RFC.best_score_

############ Plain decision trees without any hyperparameters or
# cross validatoin

from sklearn.tree import DecisionTreeClassifier
# Setup random seed
np.random.seed(42)
# Instantiate and fit the model (on the training set)
DT = DecisionTreeClassifier()
DT.fit(MinMax_scaled_X_train, Ytrain_smote.values.ravel())
# Check the score of the model (on both training and test set)
print(f"Decision Tree Accuracy on training set: {clf.score(MinMax_scaled_X_train, Ytrain_smote) * 100:.2f}%")
print(f"Decision Tree Accuracy: {clf.score(MinMax_scaled_X_test, Y_test) * 100:.2f}%")

###Check the default parameters

print('Parameters currently in use:\n')
pprint(DT.get_params())

### First apply random grid search to find a rough range
#class weight not assigned since we already upsampled
DT_random_grid = {'criterion':['gini','entropy'],
             'max_depth':[4, 5, 6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150],
             'splitter':['best','random'],
             #'min_samples_split':[1,2,'none'],
             'min_samples_leaf':[1,2,3,4,5,8,10,20,30],
             #'min_weight_fraction_leaf':[0,1,2,3],     
             #'max_features':['none','auto','log2'],
             'min_impurity_decrease':[0,1,2,3]
    
}
pprint(DT_random_grid)


# Use the random grid to search for best hyperparameters
# First create the base model to tune
DT_test = DecisionTreeClassifier()
# Random search of parameters, using 3 fold cross validation

DT_test_random = RandomizedSearchCV(estimator = DT_test, param_distributions = DT_random_grid, cv = 3, random_state=42)
# Fit the random search model
DT_test_random.fit(MinMax_scaled_X_train, Ytrain_smote.values.ravel())

DT_test_random.best_params_

##Evaluate the result of randomsearchcv

DT_best_random = DT_test_random.best_estimator_
DT_random_accuracy = evaluate(DT_best_random, MinMax_scaled_X_test, Y_test)

#### Method with cross validaiton and GRID SEARCH
tree_para = {'criterion':['gini','entropy'],
             'max_depth':[1,10,15,20,30,40,55,70,100,120],
             'splitter': ['random'],
             'min_impurity_decrease': [0],
             'min_samples_leaf': [1, 3, 5, 10, 20]
            }

DT_tuned = DecisionTreeClassifier()
grid_search_DT = GridSearchCV(estimator = DT_tuned,param_grid=tree_para,cv = 10)

grid_search_DT.fit(MinMax_scaled_X_train, Ytrain_smote.values.ravel())
grid_search_DT.best_params_


################## Check the best hyperparameter score

best_grid_DT = grid_search_DT.best_estimator_
grid_training_accuracy_DT = evaluate(best_grid_DT, MinMax_scaled_X_train, Ytrain_smote.values.ravel())
grid_accuracy_DT = evaluate(best_grid_DT, MinMax_scaled_X_test, Y_test)
grid_search_DT.best_score_

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors= 5)
neigh.fit(MinMax_scaled_X_train, Ytrain_smote.values.ravel())
print(f"K nearest Neightbour Classifier Accuracy on training set: {neigh.score(MinMax_scaled_X_train, Ytrain_smote) * 100:.2f}%")
print(f"K nearest Neightbour Classifier Accuracy: {neigh.score(MinMax_scaled_X_test, Y_test) * 100:.2f}%")

###Check the default parameters

print('Parameters currently in use:\n')
pprint(neigh.get_params())

##Using randomized grid search



###############Using other evaluation metrices




#Confusion matrix
plot_confusion_matrix(RFC, MinMax_scaled_X_test, Y_test)

#for Decision Trees
plot_confusion_matrix(DT, MinMax_scaled_X_test, Y_test)

#for K Nearest Neighbours
plot_confusion_matrix(neigh, MinMax_scaled_X_test, Y_test)

#Classification report for precision and recall
#Random Forest
from sklearn.metrics import classification_report
Y_preds_RFC = RFC.predict(MinMax_scaled_X_test)
print(classification_report(Y_test, Y_preds_RFC))

#Decision Trees
from sklearn.metrics import classification_report
Y_preds_clf = clf.predict(MinMax_scaled_X_test)
print(classification_report(Y_test, Y_preds_clf))

#K Nearest Neighbours
from sklearn.metrics import classification_report
Y_preds_neigh = neigh.predict(MinMax_scaled_X_test)
print(classification_report(Y_test, Y_preds_neigh))

Y_preds_proba_RFC = RFC.predict_proba(MinMax_scaled_X_test)

Y_preds_proba_RFC

roc_auc_score(Y_test,Y_preds_proba_RFC,multi_class="ovr")

Y_preds_proba_DT = DT.predict_proba(MinMax_scaled_X_test)

Y_preds_proba_clf

roc_auc_score(Y_test,Y_preds_proba_DT,multi_class="ovr")

Y_preds_proba_neigh = neigh.predict_proba(MinMax_scaled_X_test)

Y_preds_proba_neigh

roc_auc_score(Y_test,Y_preds_proba_neigh,multi_class="ovr")

# roc curve for classes
fpr = {}
tpr = {}
thresh ={}

n_class = 3

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(Y_test, Y_preds_proba_RFC[:,i], pos_label=i)
    
# plotting    
plt.plot(fpr[0], tpr[0], linestyle='-.',color='orange', label='Class 0 vs Rest')
plt.plot(fpr[1], tpr[1], linestyle='-.',color='green', label='Class 1 vs Rest')
plt.plot(fpr[2], tpr[2], linestyle='-.',color='blue', label='Class 2 vs Rest')
#plt.plot(fpr[3], tpr[3], linestyle='-.',color='blue', label='Class 3 vs Rest')
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
plt.savefig('Multiclass ROC',dpi=300);    

# roc curve for classes
fpr = {}
tpr = {}
thresh ={}

n_class = 3

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(Y_test, Y_preds_proba_DT[:,i], pos_label=i)
    
# plotting    
plt.plot(fpr[0], tpr[0], linestyle='--',color='orange', label='Class 0 vs Rest')
plt.plot(fpr[1], tpr[1], linestyle='--',color='green', label='Class 1 vs Rest')
plt.plot(fpr[2], tpr[2], linestyle='--',color='blue', label='Class 2 vs Rest')
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
plt.savefig('Multiclass ROC',dpi=300);    

# roc curve for classes
fpr = {}
tpr = {}
thresh ={}

n_class = 3

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(Y_test, Y_preds_proba_neigh[:,i], pos_label=i)
    
# plotting    
plt.plot(fpr[0], tpr[0], linestyle='--',color='orange', label='Class 0 vs Rest')
plt.plot(fpr[1], tpr[1], linestyle='--',color='green', label='Class 1 vs Rest')
plt.plot(fpr[2], tpr[2], linestyle='--',color='blue', label='Class 2 vs Rest')
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
plt.savefig('Multiclass ROC',dpi=300);    

##########################################
#Now checking whether outliers will have impact on the accuracy of the models

# check whether there's any significant difference if we do not remove any outliers
FILE_NAME = "data.csv"
df0 = pd.read_csv(FILE_NAME)
#still remove all the 0s in gender(as they are illegal entries, also would affect over sample technique)
gender_mode = df0.gender.mode() #gender can only be 1,2,3 not 0
df0.gender = np.where(df0.gender == 0, gender_mode, df0.gender)

ohe = OneHotEncoder(sparse=False)
ohe.fit_transform(df0[['gender']])

target = "gender"
y0 = df0[target]
x0 = df0[['affiliative', 'selfenhancing', 'agressive', 'selfdefeating', 'age','accuracy']]

X_train1, X_test1, Y_train1, Y_test1 = train_test_split(x0, y0, test_size=0.2)

oversample = SMOTE()
X_smote1_train, Y_smote_train1 = oversample.fit_resample(X_train1.astype('int'),Y_train1)



#Here we are going to try a varity of scalers, since min max might be affect by outliers now

MinMaxScaler = preprocessing.MinMaxScaler(feature_range=(0,1))
MinMax_scaled_Xtrain1 = MinMaxScaler.fit_transform(X_smote1_train)
MinMax_scaled_Xtest1 = MinMaxScaler.fit_transform(X_test1)
#MinMax_scaled_data1 = pd.DataFrame(MinMax_scaled1,columns = X_smote1.columns.values,index = X_smote1.index)
#MinMax_scaled_data1

np.random.seed(42)
RFC1 = RandomForestClassifier()
RFC1.fit(MinMax_scaled_Xtrain1,Y_smote_train1.values.ravel())
# Check the score of the model (on the test set)
print(f"Random Forest Classifier Accuracy on training set: {RFC1.score(MinMax_scaled_Xtrain1, Y_smote_train1) * 100:.2f}%")
print(f"Random Forest Classifier Accuracy: {RFC1.score(MinMax_scaled_Xtest1, Y_test1) * 100:.2f}%")
plot_confusion_matrix(RFC1, MinMax_scaled_Xtest1, Y_test1)

np.random.seed(42)
DT0 = DecisionTreeClassifier(max_depth = 3)
DT0.fit(MinMax_scaled_Xtrain1,Y_smote_train1.values.ravel())
print(f"Decision Tree Accuracy on training set: {DT0.score(MinMax_scaled_Xtrain1, Y_smote_train1) * 100:.2f}%")
print(f"Decision Tree Accuracy: {DT0.score(MinMax_scaled_Xtest1, Y_test1) * 100:.2f}%")
plot_confusion_matrix(DT0, MinMax_scaled_Xtest1, Y_test1)

neigh1 = KNeighborsClassifier(n_neighbors= 3)
neigh1.fit(MinMax_scaled_Xtrain1,Y_smote_train1.values.ravel())
print(f"K nearest Neightbour Classifier Accuracy on traning set: {neigh1.score(MinMax_scaled_Xtrain1, Y_smote_train1) * 100:.2f}%")
print(f"K nearest Neightbour Classifier Accuracy: {neigh1.score(MinMax_scaled_Xtest1, Y_test1) * 100:.2f}%")
plot_confusion_matrix(neigh1, MinMax_scaled_Xtest1, Y_test1)

#####################
#Check whether or not without any up sampling would affect the accuracy

